In [1]:
import cryo
import polars as pl
import binascii
import web3
import json
from eth_abi import decode
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd

In [2]:
MULTICALL3_ADDRESS = '0xcA11bde05977b3631167028862bE2a173976CA11'
AAVE_POOL_DATA_PROVIDER_ADDRESS = "0x7B4EB56E7CD4b454BA8ff71E4518426369a138a3"
RATE_STRATEGY_V2= "0x9ec6F08190DeA04A54f8Afc53Db96134e5E3FdFB"
DAI_ADDRESS = "0x6b175474e89094c44da98b954eedeac495271d0f"  # DAI on Ethereum Mainnet
WETH_ADDRESS = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'  # WETH
USDC_ADDRESS = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'   # USDC

In [3]:
MULTICALL3_ABI=json.loads('[{"inputs":[{"internalType":"bool","name":"requireSuccess","type":"bool"},{"components":[{"internalType":"address","name":"target","type":"address"},{"internalType":"bytes","name":"callData","type":"bytes"}],"internalType":"struct Multicall3.Call[]","name":"calls","type":"tuple[]"}],"name":"tryAggregate","outputs":[{"components":[{"internalType":"bool","name":"success","type":"bool"},{"internalType":"bytes","name":"returnData","type":"bytes"}],"internalType":"struct Multicall3.Result[]","name":"returnData","type":"tuple[]"}],"stateMutability":"payable","type":"function"}]')
AAVE_POOL_DATA_PROVIDER_ABI = json.loads('[{"inputs":[{"internalType":"address","name":"asset","type":"address"}],"name":"getReserveData","outputs":[{"internalType":"uint256","name":"unbacked","type":"uint256"},{"internalType":"uint256","name":"accruedToTreasuryScaled","type":"uint256"},{"internalType":"uint256","name":"totalAToken","type":"uint256"},{"internalType":"uint256","name":"totalStableDebt","type":"uint256"},{"internalType":"uint256","name":"totalVariableDebt","type":"uint256"},{"internalType":"uint256","name":"liquidityRate","type":"uint256"},{"internalType":"uint256","name":"variableBorrowRate","type":"uint256"},{"internalType":"uint256","name":"stableBorrowRate","type":"uint256"},{"internalType":"uint256","name":"averageStableBorrowRate","type":"uint256"},{"internalType":"uint256","name":"liquidityIndex","type":"uint256"},{"internalType":"uint256","name":"variableBorrowIndex","type":"uint256"},{"internalType":"uint40","name":"lastUpdateTimestamp","type":"uint40"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"asset","type":"address"}],"name":"getReserveTokensAddresses","outputs":[{"internalType":"address","name":"aTokenAddress","type":"address"},{"internalType":"address","name":"stableDebtTokenAddress","type":"address"},{"internalType":"address","name":"variableDebtTokenAddress","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getAllReservesTokens","outputs":[{"components":[{"internalType":"string","name":"symbol","type":"string"},{"internalType":"address","name":"tokenAddress","type":"address"}],"internalType":"struct IPoolDataProvider.TokenData[]","name":"","type":"tuple[]"}],"stateMutability":"view","type":"function"}]')
RATE_STRATEGY_V2_ABI = json.loads('[{"inputs":[{"internalType":"address","name":"reserve","type":"address"}],"name":"getInterestRateData","outputs":[{"components":[{"internalType":"uint256","name":"optimalUsageRatio","type":"uint256"},{"internalType":"uint256","name":"baseVariableBorrowRate","type":"uint256"},{"internalType":"uint256","name":"variableRateSlope1","type":"uint256"},{"internalType":"uint256","name":"variableRateSlope2","type":"uint256"}],"internalType":"struct IDefaultInterestRateStrategyV2.InterestRateDataRay","name":"","type":"tuple"}],"stateMutability":"view","type":"function"} ]')

In [4]:
# Funct Sig 4 Bytes
getAllReservesTokens_4b='b316ff89'
getReserveData_4b='35ea6a75'
getReserveTokensAddresses_4b='d2493b6c'
getIntRateData_4b='131e889c'

In [5]:
# web3 instance, function from web3py
w3 = web3.Web3()
a3 = w3.eth.contract(address = AAVE_POOL_DATA_PROVIDER_ADDRESS, abi=AAVE_POOL_DATA_PROVIDER_ABI)
m3 = w3.eth.contract(address = MULTICALL3_ADDRESS, abi=MULTICALL3_ABI)

In [38]:
tokens = [
    '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',  # WETH
    '0x6B175474E89094C44Da98b954EedeAC495271d0F',  # DAI
    '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'   # USDC
]

In [7]:
calls = []
for token in tokens:
    # Encode the call to getReserveData for this token
    calldata = a3.encode_abi("getReserveData", args=[token])
    calls.append([AAVE_POOL_DATA_PROVIDER_ADDRESS, calldata])

In [8]:
calldata_aave_pool = m3.encode_abi("tryAggregate", args=[False, calls])

In [9]:
cryo_kwargs = {
    'rpc': 'https://eth.merkle.io',
    'blocks': ['latest'], 
}
            
eth_call_df = cryo.collect(
    'eth_calls',
    to_address = [MULTICALL3_ADDRESS],
    call_data=[calldata_aave_pool],
     output_format="polars",
    **cryo_kwargs,
)

In [10]:
calldata = eth_call_df['output_data'][0]
decoded_data = decode(['(bool,bytes)[]'] , calldata)[0]

### decoded_data: 
((True,  b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x8e\xc3\xa7\xb0[K\xb1\x91\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\xd2\x19\x1c\xcaO\xb6\xb7R\xf2\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x93H\x90\xe8T\xf8U\xd6>\xf3\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0f\xca[\xcd\xac\xda+\xac[\xa6\xdd\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x15x\x88\x1f\xc6\xbd\xb0,\xa4\xc2\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03]\xf0M\\4\xe5\x0f\xbb\xab\xb0\xab\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03r#\xf2]U\xa6\xc3\x92LZh\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00g\xdd\xc7['),
 (True,
  b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x019S{%\xc3\xc0\x91FN\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x8cl\x04UWDv\xb6\xaf\x9ef\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00h\xd1\x07\x03\xcb+\xf6\xd3w&\xac\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x18kt\x18.$&\xedl\x97\x9f\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00+\x9f)j\x16\xa8\x9a\x99T\x95G\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x9f3&g\xd9\x04s\xf7i\xc2\t\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xce\x04\xbaCo\x1bc\xd4\xa0\xe0\xf6\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00g\xdd\xb9\xcf'),
 (True,
  b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x06\xeel>\xaf\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\t\xe3\x1d\rE.\xdf\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x06\xde\xa0\xcdi\xe8\xc7\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x19d#\xa5!`'k\x11Ay\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00(\x9a\xf6\xd7\t\x19\xe9\x82\xfc\xd2,\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xa1\xcf\xc6Dv\x86\xcf^\xab\x8d\xe9\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xc2A\x8a\x85s\x99\xf0\xd9\xa4\x8f\xd9\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00g\xdd\xc7+"))

In [14]:
token_data = {
    'WETH_POOL': [],
    'DAI_POOL': [],
    'USDC_POOL': []
}

####  WETH Pools Data

In [15]:
weth_reserve_data = decoded_data[0]
weth_data = decode(['int256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint40'],weth_reserve_data[1])
token_data['WETH_POOL'] = list(weth_data)

####  DAI Pools Data

In [18]:
dai_reserve_data = decoded_data[1]
dai_data=decode(['int256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint40'],dai_reserve_data[1])
token_data['DAI_POOL'] = list(dai_data)

####  USDC Pools Data

In [19]:
usdc_reserve_data = decoded_data[2]
usdc_data=decode(['int256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint40'],usdc_reserve_data[1])
token_data['USDC_POOL'] = list(usdc_data)

#### token_data
{'WETH_POOL': [0,
  28733994423441666449,
  2201086024177321944216319,
  0,
  1904452299858935736057587,
  19089498797930168402552541,
  25956637236771619518857736,
  0,
  0,
  1042019925687565315922964651,
  1066442326564534184835832424,
  1742587739],
 'DAI_POOL': [0,
  5779846338321052681806,
  169759710262633393839644262,
  0,
  126715389235410247481763500,
  29521654449100557988435871,
  52735430475195149651842375,
  0,
  0,
  1120915783932679549792928265,
  1177516073724110643260350710,
  1742584271],
 'USDC_POOL': [0,
  29769875119,
  2782988706590431,
  0,
  1933632082667719,
  30696039673592686629175673,
  49088830616933708561437228,
  0,
  0,
  1124073273672549781388103145,
  1163296147557540854013136857,
  1742587691]}

### Aave Pool DataFrame

In [22]:
field_names = [
    'unbacked', 'accruedToTreasuryScaled', 'totalAToken', 'totalStableDebt', 
    'totalVariableDebt', 'liquidityRate', 'variableBorrowRate', 'stableBorrowRate', 
    'averageStableBorrowRate', 'liquidityIndex', 'variableBorrowIndex', 'lastUpdateTimestamp'
]

In [23]:
aave_pooldata_df = pd.DataFrame(token_data, index=field_names)

In [24]:
# Convert token amounts - DAI and WETH (18 decimals)
aave_pooldata_df.loc['totalAToken', 'WETH_POOL'] = aave_pooldata_df.loc['totalAToken', 'WETH_POOL'] / 1e18
aave_pooldata_df.loc['totalAToken', 'DAI_POOL'] = aave_pooldata_df.loc['totalAToken', 'DAI_POOL'] / 1e18
aave_pooldata_df.loc['totalVariableDebt', 'WETH_POOL'] = aave_pooldata_df.loc['totalVariableDebt', 'WETH_POOL'] / 1e18
aave_pooldata_df.loc['totalVariableDebt', 'DAI_POOL'] = aave_pooldata_df.loc['totalVariableDebt', 'DAI_POOL'] / 1e18
# Convert token amounts - USDC (6 decimals)
aave_pooldata_df.loc['totalAToken', 'USDC_POOL'] = aave_pooldata_df.loc['totalAToken', 'USDC_POOL'] / 1e6
aave_pooldata_df.loc['totalVariableDebt', 'USDC_POOL'] = aave_pooldata_df.loc['totalVariableDebt', 'USDC_POOL'] / 1e6
# Convert interest rates from ray (10^27) to percentages
aave_pooldata_df.loc['liquidityRate'] = (aave_pooldata_df.loc['liquidityRate'] / 1e27)
aave_pooldata_df.loc['variableBorrowRate'] = (aave_pooldata_df.loc['variableBorrowRate'] / 1e27)

In [26]:
# Format timestamps (this does convert to strings, but that's probably what you want for timestamps)
from datetime import datetime
aave_pooldata_df.loc['lastUpdateTimestamp'] = aave_pooldata_df.loc['lastUpdateTimestamp'].apply(
    lambda x: datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S')
)

In [32]:
def display_formatted_df(aave_pooldata_df):
    # Make a copy to avoid modifying the original
    display_df = aave_pooldata_df.copy()
    
    # Format token amounts with commas and 2 decimal places
    display_df.loc['totalAToken'] = display_df.loc['totalAToken'].apply(lambda x: f"{x:,.2f}")
    display_df.loc['totalVariableDebt'] = display_df.loc['totalVariableDebt'].apply(lambda x: f"{x:,.2f}")
    
    return display_df

In [33]:
display_formatted_df(aave_pooldata_df)

,WETH_POOL,DAI_POOL,USDC_POOL
unbacked,0,0,0
accruedToTreasuryScaled,28733994423441666449,5779846338321052681806,29769875119
totalAToken,"2,201,086.02","169,759,710.26","2,782,988,706.59"
totalStableDebt,0,0,0
totalVariableDebt,"1,904,452.30","126,715,389.24","1,933,632,082.67"
liquidityRate,0.019089,0.029522,0.030696
variableBorrowRate,0.025957,0.052735,0.049089
stableBorrowRate,0,0,0
averageStableBorrowRate,0,0,0
liquidityIndex,1042019925687565315922964651,1120915783932679549792928265,1124073273672549781388103145


In [35]:
aave_pooldata_df.loc['variableBorrowRate']

WETH_POOL    0.025957
DAI_POOL     0.052735
USDC_POOL    0.049089
Name: variableBorrowRate, dtype: object

### Aave InterestRate Strategy

In [40]:
a3_2 = w3.eth.contract(address = RATE_STRATEGY_V2, abi=RATE_STRATEGY_V2_ABI)

In [41]:
calls = []
for token in tokens:
    # Encode the call to getReserveData for this token
    calldata = a3_2.encode_abi("getInterestRateData", args=[token])
    calls.append([RATE_STRATEGY_V2, calldata])

In [43]:
calldata_strategy = m3.encode_abi("tryAggregate", args=[False, calls])

In [47]:
cryo_kwargs = {
    'rpc': 'https://eth.merkle.io',
    'blocks': ['latest'], 
}
            
eth_call_strategy_df = cryo.collect(
    'eth_calls',
    to_address = [MULTICALL3_ADDRESS],
    call_data=[calldata_strategy],
     output_format="polars",
    **cryo_kwargs,
)

In [49]:
strategy_calldata = eth_call_strategy_df['output_data'][0]
strategy_decoded_data = decode(['(bool,bytes)[]'] , strategy_calldata)[0]

In [51]:
token_strategy_data = {
    'WETH_STRATEGY': [],
    'DAI_STRATEGY': [],
    'USDC_STRATEGY': []
}

In [60]:
weth_strategy_data = strategy_decoded_data[0]
weth_strategy_data
# decode(['uint16', 'uint32', 'uint32', 'uint32'],weth_strategy_data[1])

(True,
 b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\xe8vi\xc3\x08sj\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x16Ux\xee\xcf\x9d\x0f\xfb\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x95\xbe\x96\xe6@f\x97 \x00\x00\x00')

In [61]:
weth_strategy_data = strategy_decoded_data[0]
weth_strategy = decode(['uint256', 'uint256', 'uint256', 'uint256'],weth_strategy_data[1])
token_strategy_data['WETH_STRATEGY'] = list(weth_strategy)

In [63]:
dai_strategy_data = strategy_decoded_data[1]
dai_strategy = decode(['uint256', 'uint256', 'uint256', 'uint256'],dai_strategy_data[1])
token_strategy_data['DAI_STRATEGY'] = list(dai_strategy)

In [64]:
usdc_strategy_data = strategy_decoded_data[2]
usdc_strategy = decode(['uint256', 'uint256', 'uint256', 'uint256'],usdc_strategy_data[1])
token_strategy_data['USDC_STRATEGY'] = list(usdc_strategy)

In [67]:
strategy_field_names = ['optimalUsageRatio', 'baseVariableBorrowRate','variableRateSlope1','variableRateSlope2']
aave_strategy_df = pd.DataFrame(token_strategy_data, index=strategy_field_names)

In [71]:
aave_strategy_decimal_df = aave_strategy_df.map(lambda x: x / 1e27)

In [72]:
aave_strategy_decimal_df

,WETH_STRATEGY,DAI_STRATEGY,USDC_STRATEGY
optimalUsageRatio,0.900,0.920,0.920
baseVariableBorrowRate,0.000,0.000,0.000
variableRateSlope1,0.027,0.065,0.065
variableRateSlope2,0.800,0.350,0.350


### Aave InterestRate Calculation